# Notebook 5 Selected topics
Before I proceed I would like to thank this year's GTAs Fred Chen and Oksana Vertsimakha. All online materials will be available till at least 30 Oct 2026. 

### 0. Contents
Selected topics: transformation of random variables, nuisance variables, sufficient statistic, algorithms to generating random numbers (inverse transform and rejection method), Monte-Carlo integration, Markov chain, Markov chain Monte-Carlo. 

Moment estimators. Random effect model (integrated likelihood). Principal component analysis. Linear discriminant analysis. 

### 1. Selected topics in probability and statistics
Below I curated a list of special topics in probability and statistics that the previous CMEE students found them useful. They should satisfy your curiousity as a data scientist. These topics can be studied in any order. 

#### 1.1 Transformation of random variables
Let $X$ be a r.v. with pdf $f_X(x)$. $Y=g(X)$ is also a r.v.. If $g$ is a one-to-one onto function then the pdf of $Y$ is 
$$f_Y(y)=f_X(g^{-1}(y))|\frac{d}{dy}g^{-1}(y)|$$

There is a multivariate analog which involes partial derivatives (Jacobian matrix). 

#### 1.2 Nuisance variables
Given two r.v. $Y$ and $U$. Assume we know $Y|U$ has pdf $f_{Y|U}(y|u)$. $U$ is another r.v. with pdf $f_U(u|\theta)$. $\theta$ is a paramter (number). The goal is to find the distributin of $Y$ without referencing to the intermediate r.v. $U$. Here $U$ is regarded as a nuisance or latent variable. 
$$f_Y(y|\theta)=\int f_{Y|U}(y|u)f_U(u|\theta)du$$

#### 1.3 Sufficient statistic
A sufficient statistic (note: statistic is a number, statistics is a subject) summarises all information provided by the observations towards a parameter. It is the core principle of data reduction. For example in estimating $\sigma^2$ from $n$ i.i.d. $N(\mu, \sigma^2)$ samples we only need to know $\sum{x_i}/n$ and $\sum{x_i^2}/n$. There is no need to go through individual values. Therefore the two are sufficient statistics of $\sigma^2$. 

#### 1.4 Generating random numbers via inverse transform
Suppose our goal is to generate some random numbers that follow a specific distribution, say, $X\sim f_X(x)$, but we only have the uniform $U(0, 1)$ generator. The algorithm is as follows: 
1) find $F_X$, the cdf of $X$
2) find $F_X^{-1}$, the inverse of $F_X$
3) sample $u\sim U(0, 1)$, then $F_X^{-1}(u)$ follows the intended $f_X$

The challenge is to find the inverse of a function. 

#### 1.5 Generating random numbers via rejection algorithm
Suppose we want to generate $X\sim f_X(x)$, but we only know how to generation random numbers from another distribution $Y\sim f_Y(y)$. We call it the proposal distribution. Note that the support of $Y$ to at least cover the support of $X$. The algorithm is as follows:
1) find a constant number $M$, such that $f_X(x)<=M*f_Y(x)$ for all $x$ within its support. In other words, plot the pdfs of $X$ and $Y$ on the same graph, rescale $f_Y$ with a multiplyer $M$ such that $M*f_Y$ envelopes $f_X$. 
2) sample $y\sim f_Y$
3) sample $u\sim U(0, 1)$
4) if $u<\frac{f_X(y)}{M*f_Y(y)}$ then accept this $y$, discard otherwise

Those accepted will follow $f_X$. 

#### 1.6 Monte-Carlo integration
Suppose our goal is to evaluate the following integral: 
$$I=\int_0^1\sqrt{1-x^2}dx$$
You can calculate the integral by hand (!), or by quadrature rule (counting rectangles). For instance, via <code>integrate()</code> in R:

In [2]:
integrate(function(x){sqrt(1-x^2)}, lower=0, upper=1)

0.7853983 with absolute error < 0.00011

Note that <code>integrate()</code> is deterministic. Since we know how to draw random numbers from R, here I propose an alternative approach for the same problem: 
1) sample $n$ numbers from $U(0, 1)$
2) compute $I_n=\frac{1}{n}\sqrt{1-x_i^2}$
3) If $n$ is sufficiently large then $I_n$ is a good approximation to the original integral $I$

In general, if we want to find $I=\int_a^bg(x)dx$, it is equivalent to finding
$$I=\int_a^bg(x)dx=\int_a^b\frac{g(x)}{f(x)}f(x)dx$$
$$=E_X[\frac{g(X)}{f(X)}]$$
the expectation of the transformed r.v. $g(X)/f(X)$, where $X$ has pdf $f$. Note that the support of the proposed r.v. $X$ must include $(a,b)$. Remember expectation is the hypothetical average of infinitely many trials, which is "achievable" by computer. This also justifies the Monte-Carlo simulations we did in the Evolutionary Modelling week. 

I hope you appreciate the simplicity of MC methods. It is also a beauty to use some random numbers to approximate a fixed, deterministic quantity. MC integration works for $\pm\infty$ bounds as long as a suitable r.v. is chosen (e.g. normal, t-distribution, or exponential between 0 and $\infty$). 
Of course, the stochastic nature of Monte Carlo integration means there will be errors, and convergence can be slow for multiple integrals (with multivariate r.v.). 

#### 1.7 Markov Chain
Markov Chain is a chain of time-dependent r.v., $X(t), X(t+1), X(t+2), ...$. There are several states (possible outcomes, discrete or continuous) that $X(j)$ can take on. For example, the allele count of a locus over successive generations, whose states can be ${0, 1, 2, ..., 2N}$ for a population of $N$ diploids. Another example is the size of a population in a birth and death process. When there is a birth event, the population size moves up to the next value (state), and down by one when a death event occurs. 

The chain transits from one state to another by chance over time, and the transition probabilites depend only on the current state. Often the probabilities are put in a Markov matrix $M$ with number of rows and columns equal the number of states. A time-homogeneous Markov chain is a special case whose transition probabilties remain the same over time. 

The Wright-Fisher model can be analysed as a time-homogeneous Markov Chain. For example, with $N=2$, there are 5 states for the allele counts ${0, 1, 2, 3, 4}$. Its Markov matrix is: 

In [5]:
WF<-function(N)
{
    k<-seq(0, 2*N, 1)
    f<-function(x, y) {dbinom(y, size=2*N, prob=x/(2*N))}
    return(outer(k, k, f))
}
WF(2) # WRIGHT-FISHER MAXTIX WITH N=2. 5 STATES: 0, 1, 2, 3, 4 COPIES OF ALLELES

1.00000000,0.000000,0.0000000,0.000000,0.00000000
0.31640625,0.421875,0.2109375,0.046875,0.00390625
0.06250000,0.250000,0.3750000,0.250000,0.06250000
0.00390625,0.046875,0.2109375,0.421875,0.31640625
0.00000000,0.000000,0.0000000,0.000000,1.00000000


The one-step transition probability can be found from the elements of $\boldsymbol{M}$. For instance, the probability of having 4 copies of an allele in the next generation given there is only 1 currently is 0.00390625 (second row, last element). In the WF matrix, the first and the last rows are the absorbing states where you are stuck once you have entered. These two refer to the cases of extinction (0 copy) and fixation (4 copies) of an allele. 

Let us study $\boldsymbol{M}$ more closely. First, $\boldsymbol{M}$ has to be non-negative, and rows must sum to one. Some $\boldsymbol{M}$ may have a limiting distribution, that $\lim_{T\rightarrow\infty}\boldsymbol{M}^T$ exists. Some may even have a stationary distribution $\pi$, where $\pi \boldsymbol{M}=\pi$. 

#### 1.8 Markov Chain Monte-Carlo integration
In #1.6 we used a sequence of i.i.d. random numbers ${x_1, x_2, ..., x_n}$ to approximate an integral. That is, 
$$\int_a^b g(x)dx\approx\frac{1}{n}\sum_{i=1}^n\frac{g(x_i)}{f(x_i)}$$
In fact, the approximation still holds for *dependent* numbers. In some (usually multi-dimensional) cases, it is simpler to construct a Markov chain, whose stationary distribution is $f$, from which Monte-Carlo samples are drawn. These MCMC (Markov chain + Monte-Carlo) algorithms include Gibbs sampling and Metropolis-Hasting (MH), which are to be introduced in the Bayesian module. 

### 2. Moment estimators
Although the MLE is the best method to find estimators it often fails to provide closed-form answers. One example is gamma samples, where the exact MLE cannot be derived by hand even the log-likelihood is known. Let $X_1, X_2, ..., X_n\sim Gamma(\alpha, \beta)$, i.i.d.. The log-likelihood function for the two paramters is:
$$l(\alpha, \beta)=n\alpha\ln(\beta)-n\ln(\Gamma(\alpha))+(\alpha-1)\sum\ln(x_i)-\beta\sum{x_i}$$
where $\Gamma(\alpha)$ is the gamma function. The derivatives of the gamma function often involves the digamma function which cannot be evaluated explicitly. As a result, the MLE $(\hat{\alpha}, \hat{\beta})$ can only be evaluated numerically, say, via <code>optim()</code>. 

There exist other estimators such as the method of moments. It matches population moments with sample moments. In the example of gamma, the first two moments should suffice for the estimation of $\alpha$ and $\beta$. The first two population moments of a gamma r.v. are:
$$E[X_i]=\frac{\alpha}{\beta}$$
$$E[X_i^2]=\frac{\alpha(\alpha+1)}{\beta^2}$$
These moments can be derived from its pdf or mgf. Now we turn to the sample moments, which come from our observations: 
$$m_1=\frac{\sum_{i=1}^n{x_i}}{n}$$
$$m_2=\frac{\sum_{i=1}^n{x_i^2}}{n}$$
The method of moments matches the population moments with the sample moments:
$$\{\begin{array}{c} \frac{\alpha}{\beta}=m_1 \\ \frac{\alpha(\alpha+1)}{\beta^2}=m_2\end{array}$$
The remaining task is to find the pair of $(\tilde{\alpha}, \tilde{\beta})$ that satisfies this system of equations, and they are the moment estimators. Note that they do not necessarily enjoy the five properties of ML estimators we discussed on Day 3. 

### 3. Random effect model
In linear models (or GLMs) the explanatory variables are *fixed* effects as they are factors that we have control on or would like to investigate. In contrast, *random* effects have factor levels that are drawn from a large pool in which the individuals vary in many ways, but we do not know exactly how or why they differ. 

A simple linear random effect model looks like this:
$$y_{ij}=\mu+\tau_j+\epsilon_{ij}$$
$$i=1,2,...,n, j=1,2,..., k$$
In this model there are $k$ random groups each with $n$ observations. $\mu$ is the overall mean, and $\epsilon_{ij}\sim N(0, \sigma^2)$ is the i.i.d. individual error term. In addition, $\tau_j\sim N(0, \sigma_{random}^2)$ is another random error term assigned to all members within group $j$. By definition the likelihood it is the joint pdf of all $k*n$ observations: 
$$L(\underline{\theta})=f(y_{11}, ..., y_{n1}, y_{12}, ..., y_{n2}, ..., y_{1k}, ..., y_{nk})$$
It is important to understand that individuals from the same random group are *not independent* as they share the same value of $\tau_j$. Therefore
$$L(\underline{\theta})=f_{group_1}(y_{11}, ..., y_{n1})f_{group_2}(y_{12}, ..., y_{n2})...f_{group_k}(y_{1k}, ..., y_{nk})$$
where $f_{group_j}$ is the joint pdf of the observations from the random group $j$. We cannot simplify the likelihood further. 

If we zoom into the joint pdf of group $j$, 
$$f_{group_j}(y_{1j}, ..., y_{nj})=\int f_{group_j}(y_{1j}, ..., y_{nj}|\tau_j)f(\tau_j)d\tau_j$$
by treating $\tau_j$ as a nuisance variable (see #1.3 above). In fact, $Y_{ij}|\tau_j\sim N(\mu+\tau_j, \sigma^2)$. As a result, the likelihood is a product of integrals and often can only be evaluated numerically. There are algorithms to approximate integrals (e.g. Laplace approximation), as implemented in <code>lmer()</code> or <code>glmer()</code>. 

### 4. Supervised and unsupervised learning 
Machine learning?
#### 4.1 Principal component analysis, data reduction, unsupervised learning
Data reduction may be required when too many (correlated) variables are collected, especially when the number of variables exceeds the number of individuals. Think of a whole-genome dataset, say. 

With hundreds of thousands of loci it becomes infeasible to visualise the dataset, let alone make some sense out of it. One may think whether it is possible to get rid of some variables, or to create a few new variables to represent the data while preserving most variation. PCA is an algorithm to create principal components which are linear combinations of the existing variables. The components are then ranked from the most important the least by the amount of variance explained. If you are lucky, the first few compoments may capture a good proportion of the total variation for downstream analyses. In R, we have <code>prcomp()</code> to perform PCA, or <code>eigen()</code> for eigen-decomposition of the variance-covariance matrix. 

#### 4.2 Linear discriminant analysis
PCA is unsupervised as no labels are involved (i.e. we don't know the individuals behind the genotypes). There are scenarios when the labels are available, for example, if the species or the populations of the individuals are known. By combining the labels and dataset, we can use LDA to create discrimiants, which are again linear combiantions of the variables, to maximise the separation of classes. Since labels are provided it is a supervised learning algorithm. 

Note: we can also use logistic or multinomial logistic regression for supervised classification. 